<span style = 'color:red;font-size:25px'>提取相同的MMSI数据

In [ ]:
import os
import pandas as pd

# 定义输入文件路径
input_files = [
                             # 数据地址
]

# 定义输出文件夹路径
output_folder = r""

# 确保输出文件夹存在
os.makedirs(output_folder, exist_ok=True)

# 初始化存储所有 MMSI 数据的字典
mmsi_data = {}

# 遍历每个输入文件并按 MMSI 号分类
for file in input_files:
    try:
        # 读取 CSV 文件
        df = pd.read_csv(file)
        if 'MMSI' not in df.columns:
            print(f"文件 {file} 不包含 MMSI 列，跳过...")
            continue  # 如果文件没有 MMSI 列则跳过

        # 按 MMSI 分组并保存到字典中
        for mmsi, group in df.groupby('MMSI'):
            if mmsi not in mmsi_data:
                mmsi_data[mmsi] = group
            else:
                mmsi_data[mmsi] = pd.concat([mmsi_data[mmsi], group], ignore_index=True)
    except Exception as e:
        print(f"处理文件 {file} 时出错: {e}")

# 保存每个 MMSI 的数据到单独的 CSV 文件
for mmsi, data in mmsi_data.items():
    output_file = os.path.join(output_folder, f"{mmsi}.csv")
    data.to_csv(output_file, index=False)

print("数据提取和保存完成！")

<span style = 'color:red;font-size:25px'>去除异常值点

In [ ]:
import os
import pandas as pd

# 指定文件夹路径
folder_path = r"  "

# 获取文件夹中的所有CSV文件
csv_files = [file for file in os.listdir(folder_path) if file.endswith(".csv")]

# 遍历每个CSV文件
for csv_file in csv_files:
    # 构建完整的文件路径
    file_path = os.path.join(folder_path, csv_file)
    
    # 读取CSV文件
    data = pd.read_csv(file_path)
    
    # 转换 '# Timestamp' 列为 datetime
    if 'BaseDateTime' in data.columns:
        data['BaseDateTime'] = pd.to_datetime(data['BaseDateTime'], format='%Y-%m-%dT%H:%M:%S', errors='coerce')
    
        # 按时间戳升序排序
        data.sort_values('BaseDateTime', inplace=True)
    
    # 删除 'SOG' 列中小于0.5或大于30的行
    data = data[(data['SOG'] >= 0.5) & (data['SOG'] <= 30)]
    
    # 删除 'COG' 列中小于0或大于360的行
    data = data[(data['COG'] >= 0) & (data['COG'] <= 360)]
    
    # 删除 'Latitude' 列中小于-90或大于90的行
    data = data[(data['LAT'] >= -90) & (data['LAT'] <= 90)]
    
    # 删除 'Longitude' 列中小于0或大于180的行
    data = data[(data['LON'] >= 0) & (data['LON'] <= 180)]
    
    # 删除 'Heading' 列中小于0或大于180的行
    data = data[(data['Heading'] >= 0) & (data['Heading'] <= 180)]
    
    # 删除 "ROT" 列（如果需要删除时取消注释下面一行）
    # data.drop('ROT', axis=1, inplace=True)
    
    # 覆盖原CSV文件
    data.to_csv(file_path, index=False)

print("数据处理完成。")


<span style = 'color:red;font-size:25px'>填充有效的静态数据

In [ ]:
import os
import pandas as pd

# 定义目标列
target_columns = ['VesselType', 'Width', 'Length', 'Draft']

# 获取目标文件夹中的所有CSV文件
folder_path = r"   "
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

# 遍历文件夹中的每个CSV文件
for csv_file in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    
    # 读取CSV文件
    df = pd.read_csv(file_path)
    
    # 存储每列的有效值
    valid_values = {}

    # 遍历每个目标列，查找第一个有效值
    for column in target_columns:
        if column in df.columns:
            # 提取有效值（去除空值、'undefined'、'Undefined'）
            valid_value = df[column][(df[column].notna()) & ~(df[column].isin(['undefined', 'Undefined', '']))]
            
            # 直接取第一个有效值
            if not valid_value.empty:
                valid_values[column] = valid_value.iloc[0]
            else:
                valid_values[column] = None  # 如果没有有效值，设置为None

    # 用提取的有效值填充目标列
    for column in target_columns:
        if column in valid_values and valid_values[column] is not None:
            # 用提取到的有效值填充整列
            df[column] = valid_values[column]
    
    # 保存修改后的CSV文件
    df.to_csv(file_path, index=False)

    print(f"目标列已成功填充并保存到 {file_path}")

print("所有CSV文件处理完成。")


<span style = 'color:red;font-size:25px'>转换时间戳类型

In [ ]:
import os
import pandas as pd

# 指定目标文件夹路径
folder_path = "  "

# 获取目标文件夹中的所有CSV文件
csv_files = [file for file in os.listdir(folder_path) if file.endswith(".csv")]

# 遍历所有CSV文件
for csv_file in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    
    # 读取CSV文件
    data = pd.read_csv(file_path)
    
    # 检查时间戳列是否存在
    if 'BaseDateTime' in data.columns:
        # 尝试将时间戳列转换为datetime格式，遇到无法转换的就保留原格式
        try:
            data['BaseDateTime'] = pd.to_datetime(data['BaseDateTime'], format='%Y/%m/%d %H:%M:%S', errors='raise')
        except ValueError:
            # 如果发生异常，说明时间戳格式不符合要求，需要进行修改
            data['BaseDateTime'] = pd.to_datetime(data['BaseDateTime'], errors='coerce')
            # 将时间戳列格式化为目标格式
            data['BaseDateTime'] = data['BaseDateTime'].dt.strftime('%Y/%m/%d %H:%M:%S')
        
        # 保存修改后的CSV文件
        data.to_csv(file_path, index=False)

print("CSV文件中的时间戳已根据需要进行更改。")


<span style = 'color:red;font-size:25px'>分割数据

In [ ]:
import os
import pandas as pd

# 指定目标文件夹路径
folder_path = "  "

# 获取目标文件夹中的所有CSV文件
csv_files = [file for file in os.listdir(folder_path) if file.endswith(".csv")]

# 遍历所有CSV文件
for csv_file in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    
    # 读取CSV文件
    data = pd.read_csv(file_path)
    
    # 初始化数据分段的起点
    segment_start = 0
    segments = []  # 用于存储分段数据

    # 遍历数据，计算相邻时间差
    for i in range(1, len(data)):
        # 将时间戳转换为pandas的日期时间对象
        timestamp1 = pd.to_datetime(data['BaseDateTime'].iloc[i - 1], format='%Y/%m/%d %H:%M:%S')
        timestamp2 = pd.to_datetime(data['BaseDateTime'].iloc[i], format='%Y/%m/%d %H:%M:%S')
        
        # 如果转换失败，则跳过此行
        if pd.isna(timestamp1) or pd.isna(timestamp2):
            continue
        
        # 计算相邻两行的时间差，单位为小时
        timestamp_diff = (timestamp2 - timestamp1).total_seconds() / 3600  # 结果是小时

        # 如果相邻两值的时间差大于6小时，分割数据
        if timestamp_diff > 6:
            # 创建新的分段
            segment_data = data.iloc[segment_start:i]
            segments.append(segment_data)
            segment_start = i

    # 如果有分段数据
    if len(segments) > 0:
        # 保存分段数据到新的CSV文件
        for idx, segment_data in enumerate(segments):
            # 提取MMSI号作为文件名前缀
            mmsi_number = data['MMSI'].iloc[0] if 'MMSI' in data.columns else 'Unknown'
            segment_filename = f"{mmsi_number}_{idx + 1}.csv"
            segment_path = os.path.join(folder_path, segment_filename)
            segment_data.to_csv(segment_path, index=False)

        # 从原文件中删除与保存数据中相同 `# Timestamp` 值的行
        for segment in segments:
            data = data[~data['BaseDateTime'].isin(segment['BaseDateTime'])]

        # 保存原文件
        data.to_csv(file_path, index=False)

print("CSV文件中的数据已根据# Timestamp分段，分段已从原文件中删除，并删除与保存数据中相同 `# Timestamp` 值的行。")

<span style = 'color:red;font-size:25px'>删除跳点以及重复时间戳的点

In [ ]:
import pandas as pd
import os
import math

def get_distance(lon1, lat1, lon2, lat2):
    """
    计算经纬度之间的距离，单位千米
    :param lon1: A点的经度
    :param lat1: A点的纬度
    :param lon2: B点的经度
    :param lat2: B点的纬度
    :return: 距离（千米）
    """
    EARTH_RADIUS = 6371  # 地球半径，单位千米
    lon1, lat1, lon2, lat2 = map(math.radians, [float(lon1), float(lat1), float(lon2), float(lat2)])
    d_lon = lon2 - lon1
    d_lat = lat2 - lat1
    a = math.sin(d_lat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(d_lon / 2) ** 2
    c = 2 * math.asin(math.sqrt(a))
    distance = c * EARTH_RADIUS
    return distance

def clear_list(gjd_list=None, max_speed=56, time_interval=6):
    """
    清理重复时间戳并删除跳点
    删除时间戳重复的数据，保留时间戳第一次出现的数据
    删除跳点：基于相邻轨迹点的距离与时间差，超出阈值则删除
    :param time_interval: 时间分割阈值，相邻两点超过特定值，此处为6h，以此分割轨迹段
    :param max_speed: 正常速度阈值，相邻两点平均速度超过设定值(此处为56Km/h)，删除异常速度的轨迹点
    :param gjd_list: 轨迹点列表
    :return: 清理后的索引列表
    """
    if not gjd_list:
        return '传入列表为空'

    # 按照时间去重
    timestamp_list = []
    repeat_timestamp = []
    for gjd_index, gjd in enumerate(gjd_list):
        gjd_timestamp = gjd[0]  # 时间戳在第0列
        if gjd_timestamp in timestamp_list:
            repeat_timestamp.append(gjd_index)  # 记录重复的索引
        else:
            timestamp_list.append(gjd_timestamp)

    # 只保留未重复的行
    valid_indices = [i for i in range(len(gjd_list)) if i not in repeat_timestamp]

    # 处理跳点
    final_indices = []  # 存储最终有效的轨迹点索引
    for idx in valid_indices:
        if idx + 1 >= len(gjd_list):  # 如果是最后一个点
            final_indices.append(idx)
            break

        current_point = gjd_list[idx]  # 当前轨迹点
        next_point = gjd_list[idx + 1]  # 下一个轨迹点

        # 解析当前点和下一点的数据
        lon1, lat1, timestamp1 = float(current_point[2]), float(current_point[1]), pd.to_datetime(current_point[0])
        lon2, lat2, timestamp2 = float(next_point[2]), float(next_point[1]), pd.to_datetime(next_point[0])

        # 计算两点之间的距离和时间差
        distance = get_distance(lon1, lat1, lon2, lat2)  # 单位千米
        time_diff = (timestamp2 - timestamp1).total_seconds() / 3600  # 单位小时

        # 判断是否为跳点
        if time_diff == 0 or (distance / time_diff) > max_speed:
            continue  # 跳过跳点
        else:
            final_indices.append(idx)

    # 返回清理后的索引
    return final_indices

# 文件处理
target_folder = r"F:\MMSI数据(加州)"  # 文件夹路径

# 获取文件夹中的所有CSV文件
csv_files = [file for file in os.listdir(target_folder) if file.endswith(".csv")]

for csv_file in csv_files:
    file_path = os.path.join(target_folder, csv_file)
    try:
        # 读取CSV文件
        data = pd.read_csv(file_path)
        
        # 提取时间戳、纬度和经度
        gjd_list = data[['BaseDateTime', 'LAT', 'LON']].values.tolist()

        # 清理数据并返回有效的行索引
        valid_indices = clear_list(gjd_list)
        
        # 根据有效行索引提取原数据中的行
        cleaned_data = data.iloc[valid_indices]

        # 将清理后的数据覆盖保存到原文件中
        cleaned_data.to_csv(file_path, index=False)
        # print(f"文件 {csv_file} 处理完成。")
    except Exception as e:
        print(f"文件 {csv_file} 处理出错：{e}")

print("所有文件处理完成。")

<span style = 'color:red;font-size:25px'>提取经纬度不在特定范围的数据

In [ ]:
import os
import pandas as pd

# 1. 定义经纬度范围（将度、分、秒转换为十进制度数）

def dms_to_decimal(degrees, minutes, seconds):
    return degrees + minutes / 60 + seconds / 3600

# 区域二的纬度范围
lat_max = dms_to_decimal(27, 37, 19.793)    # 27°37′19.793″
lat_min = dms_to_decimal(25, 2, 57.534)     # 25°2′57.534″

# 区域二的经度范围
lon_max = dms_to_decimal(80, 30, 8.39)      # 80°30′8.39″
lon_min = dms_to_decimal(78, 16, 39.534)    # 78°16′39.534″

# 2. 文件夹路径
target_folder = r"  "

# 3. 初始化一个列表，用于存储超出范围的文件名和详细信息
outside_range_files = []

# 4. 获取文件夹中的所有 CSV 文件
csv_files = [file for file in os.listdir(target_folder) if file.endswith(".csv")]

# 5. 遍历每个 CSV 文件
for csv_file in csv_files:
    file_path = os.path.join(target_folder, csv_file)
    try:
        # 读取 CSV 文件，只读取 'Latitude' 和 'Longitude' 列
        data = pd.read_csv(file_path, usecols=['LAT', 'LON'])

        # 确保 'Latitude' 和 'Longitude' 列为数值型，并去除缺失值
        data = data.dropna(subset=['LAT', 'LON'])
        data['LAT'] = pd.to_numeric(data['LAT'], errors='coerce')
        data['LON'] = pd.to_numeric(data['LON'], errors='coerce')

        # 重新去除可能的 NaN 值
        data = data.dropna(subset=['LAT', 'LON'])

        # 检查是否存在超出纬度范围的值
        lat_out_of_range = data[(data['LAT'] < lat_min) | (data['LAT'] > lat_max)]
        is_lat_out = not lat_out_of_range.empty

        # 检查是否存在超出经度范围的值
        lon_out_of_range = data[(data['LON'] < lon_min) | (data['LON'] > lon_max)]
        is_lon_out = not lon_out_of_range.empty

        # 根据情况记录文件名、超出范围的信息和具体值
        if is_lat_out or is_lon_out:
            status = ""
            details = ""
            max_display = 5  # 最多显示5个超出值
            if is_lat_out and is_lon_out:
                status = "Latitude and Longitude out of range"
                lat_values = lat_out_of_range['LAT'].values[:max_display]
                lon_values = lon_out_of_range['LON'].values[:max_display]
                details += f"Latitude out of range values (showing up to {max_display}):\n{lat_values}\n"
                details += f"Longitude out of range values (showing up to {max_display}):\n{lon_values}\n"
            elif is_lat_out:
                status = "Latitude out of range"
                lat_values = lat_out_of_range['LAT'].values[:max_display]
                details += f"Latitude out of range values (showing up to {max_display}):\n{lat_values}\n"
            else:
                status = "Longitude out of range"
                lon_values = lon_out_of_range['LON'].values[:max_display]
                details += f"Longitude out of range values (showing up to {max_display}):\n{lon_values}\n"
            outside_range_files.append((csv_file, status, details))
    except Exception as e:
        print(f"读取文件 {csv_file} 时出错：{e}")

# 6. 输出所有符合条件的 CSV 文件名、超出范围的情况和具体值
if outside_range_files:
    print("以下文件包含了超出指定经纬度范围的值：")
    for filename, status, details in outside_range_files:
        print(f"\n文件名: {filename}")
        print(f"超出范围情况: {status}")
        print(f"具体超出范围的值:\n{details}")
else:
    print("未找到包含超出指定经纬度范围的值的文件。")


# <span style = 'color:red;font-size:25px'>数据插值

In [ ]:
import os
import pandas as pd
import numpy as np
from scipy.interpolate import InterpolatedUnivariateSpline

# 指定文件夹路径
input_folder = r"   "

# 获取所有CSV文件
csv_files = [file for file in os.listdir(input_folder) if file.endswith(".csv")]

for csv_file in csv_files:
    # 构建完整的文件路径
    file_path = os.path.join(input_folder, csv_file)
    
    # 读取CSV文件
    data = pd.read_csv(file_path)
    
    # 去重数据
    data = data.drop_duplicates(subset='BaseDateTime', keep='first')

    # 对数据按照 '# Timestamp' 列进行排序
    data = data.sort_values(by='BaseDateTime')
    
    # 将 '# Timestamp' 列转换为 datetime 对象
    data['BaseDateTime'] = pd.to_datetime(data['BaseDateTime'], format='%Y/%m/%d %H:%M:%S')
    
    # 创建以1分钟为间隔的新时间戳范围
    start_time = data['BaseDateTime'].min()
    end_time = data['BaseDateTime'].max()
    new_timestamps = pd.date_range(start=start_time, end=end_time, freq='1min')  # 修改为 '1min'
    
    # 将时间戳转换为数值型（以秒为单位）用于插值
    data['Timestamp_numeric'] = data['BaseDateTime'].astype(np.int64) // 10**9
    new_timestamps_numeric = new_timestamps.astype(np.int64) // 10**9

    # 确保有足够的数据点进行插值
    if len(data) < 2:
        continue  # 数据点不足，跳过

    # 处理 'Latitude' 和 'Longitude' 列的插值
    try:
        valid_data = data.dropna(subset=['LAT', 'LON'])
        if len(valid_data) < 4:
            continue  # 数据点不足以进行插值

        # 在新时间戳上进行插值
        new_latitudes = []
        new_longitudes = []
        
        for i in range(len(new_timestamps_numeric)):
            timestamp = new_timestamps_numeric[i]
            
            # 找到相邻的时间点
            prev_idx = np.searchsorted(valid_data['Timestamp_numeric'], timestamp, side='right') - 1
            next_idx = prev_idx + 1

            # 确保有前后两个点
            if prev_idx >= 0 and next_idx < len(valid_data):
                prev_timestamp = valid_data.iloc[prev_idx]['Timestamp_numeric']
                next_timestamp = valid_data.iloc[next_idx]['Timestamp_numeric']

                time_diff = next_timestamp - prev_timestamp
                if time_diff <= 120:  # 小于或等于2分钟，使用三次样条插值
                    spline_lat = InterpolatedUnivariateSpline(valid_data['Timestamp_numeric'], valid_data['LAT'])
                    spline_lon = InterpolatedUnivariateSpline(valid_data['Timestamp_numeric'], valid_data['LON'])
                    new_latitudes.append(spline_lat(timestamp))
                    new_longitudes.append(spline_lon(timestamp))
                else:  # 大于2分钟，使用线性插值
                    lat_interp = np.interp(timestamp, valid_data['Timestamp_numeric'], valid_data['LAT'])
                    lon_interp = np.interp(timestamp, valid_data['Timestamp_numeric'], valid_data['LON'])
                    new_latitudes.append(lat_interp)
                    new_longitudes.append(lon_interp)
            else:
                new_latitudes.append(np.nan)
                new_longitudes.append(np.nan)

    except Exception as e:
        print(f"文件 {csv_file} 的 'Latitude' 和 'Longitude' 插值出错：{e}")
        continue

    # 创建新的 DataFrame，包含插值后的数据
    interpolated_data = pd.DataFrame({
        'BaseDateTime': new_timestamps,
        'LAT': new_latitudes,
        'LON': new_longitudes
    })

    # 保留 '# Timestamp' 列的格式（如 '2020/8/5  8:47:40'）
    interpolated_data['BaseDateTime'] = interpolated_data['BaseDateTime'].apply(lambda x: f"{x.year}/{x.month}/{x.day} {x.hour}:{x.minute}:{x.second}")

    # 将新时间戳转换为数值型，方便后续计算
    interpolated_data['Timestamp_numeric'] = pd.to_datetime(interpolated_data['BaseDateTime'], format='%Y/%m/%d %H:%M:%S').astype(np.int64) // 10**9

    # 添加固定列：'MMSI'、'Ship type'、'Width'、'Length'、'Draught'
    fixed_columns = ['MMSI', 'VesselType', 'Width', 'Length', 'Draft']
    for col in fixed_columns:
        if col in data.columns:
            value = data[col].iloc[0]  # 获取第一个值
            interpolated_data[col] = value
        else:
            interpolated_data[col] = np.nan  # 若列不存在，填充 NaN

    # 对 'SOG'、'COG'、'Heading' 列进行前后3-5个数据的平均值计算
    for col in ['SOG', 'COG', 'Heading']:
        if col in data.columns:
            # 准备数据
            data_col = data[['Timestamp_numeric', col]].dropna()
            data_col = data_col.sort_values('Timestamp_numeric').reset_index(drop=True)
            # 初始化列表存储平均值
            averages = []
            timestamps_numeric = interpolated_data['Timestamp_numeric'].values
            for timestamp in timestamps_numeric:
                # 找到当前插值时间点在原始数据中的位置
                idx = np.searchsorted(data_col['Timestamp_numeric'], timestamp)
                # 获取前后3-5个数据点
                start_idx = max(0, idx - 5)
                end_idx = min(len(data_col), idx + 5)
                window_data = data_col.iloc[start_idx:end_idx][col]
                if len(window_data) >= 1:
                    avg_value = window_data.mean()
                    averages.append(avg_value)
                else:
                    averages.append(np.nan)
            interpolated_data[col] = averages
        else:
            interpolated_data[col] = np.nan  # 若列不存在，填充 NaN

    # 删除辅助列
    interpolated_data.drop(columns=['Timestamp_numeric'], inplace=True)

    # 保存插值后的数据，覆盖原文件
    interpolated_data.to_csv(file_path, index=False)
    
    print(f"文件 {csv_file} 已处理并覆盖保存。")

print("所有文件处理完毕。")


<span style = 'color:red;font-size:25px'>中值滤波

In [ ]:
import os
import pandas as pd
import numpy as np
from scipy.signal import medfilt

# 指定文件夹路径
input_folder = r"F:\MMSI数据(加州)"

# 获取所有CSV文件
csv_files = [file for file in os.listdir(input_folder) if file.endswith(".csv")]

for csv_file in csv_files:
    try:
        # 构建完整的文件路径
        file_path = os.path.join(input_folder, csv_file)
        
        # 读取CSV文件
        data = pd.read_csv(file_path)
        
        # 确保数据按照 '# Timestamp' 列进行排序
        data = data.sort_values(by='BaseDateTime').reset_index(drop=True)
        
        # 检查是否存在缺失值，删除包含缺失值的行
        data = data.dropna(subset=['LAT', 'LON']).reset_index(drop=True)
        
        # 设置中值滤波的窗口大小，必须为奇数
        window_size = 5  # 您可以根据需要调整窗口大小

        # 检查数据长度是否足够应用滤波
        if len(data) >= window_size:
            # 从第三个数据点开始应用中值滤波
            # 提取需要滤波的部分
            data_to_filter = data[['LAT', 'LON']].copy()

            # 应用中值滤波
            # 注意：为了从第三个数据点开始滤波，我们需要在前面填充适当数量的值
            # 在这种情况下，我们可以使用 padding='edge' 参数来处理边界效应
            data_filtered_lat = medfilt(data_to_filter['LAT'], kernel_size=window_size)
            data_filtered_lon = medfilt(data_to_filter['LON'], kernel_size=window_size)

            # 将滤波后的数据替换原始数据，从第三个数据点开始
            data.loc[2:, 'LAT'] = data_filtered_lat[2:]
            data.loc[2:, 'LON'] = data_filtered_lon[2:]
        else:
            print(f"文件 {csv_file} 数据点少于窗口大小，跳过滤波。")
        
        # 保存处理后的数据，覆盖原文件
        data.to_csv(file_path, index=False)
        
        print(f"文件 {csv_file} 已处理并覆盖保存。")
    except Exception as e:
        print(f"处理文件 {csv_file} 时出错：{e}")
        continue

print("所有文件处理完毕。")
